<a href="https://colab.research.google.com/github/defyMiy/NMT-Project/blob/main/mt5-small_5000ds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Jan 28 20:56:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
!pip install simpletransformers
!pip install torch
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# import pandas as pd

# data = pd.read_csv('/content/gdrive/MyDrive/dataset5000pair.csv', usecols=['Thai', 'English'])

# !rm -rf '/content/train.csv'
# !rm -rf '/content/eval.csv'

# from sklearn.model_selection import train_test_split

# train_data, eval_data = train_test_split(data, train_size=0.85, shuffle=True, random_state=1234)
# train_data.to_csv('/content/train.csv', index=False)
# eval_data.to_csv('/content/eval.csv', index=False)

# train_data = pd.read_csv('/content/gdrive/MyDrive/train.csv')
# eval_data = pd.read_csv('/content/gdrive/MyDrive/eval.csv')

# !rm -rf '/content/train.csv'
# !rm -rf '/content/eval.csv'

# import csv

# with open('/content/train.csv', 'w') as csv_file:
#   writer = csv.writer(csv_file)
#   writer.writerow(['prefix', 'input_text', 'target_text'])
#   for i in range(len(train_data)):
#     writer.writerow(['translate thai to english', train_data['Thai'][i], train_data['English'][i]])
#     writer.writerow(['translate english to thai', train_data['English'][i], train_data['Thai'][i]])
# with open('/content/eval.csv', 'w') as csv_file:
#   writer = csv.writer(csv_file)
#   writer.writerow(['prefix', 'input_text', 'target_text'])
#   for i in range(len(eval_data)):
#     writer.writerow(['translate thai to english', eval_data['Thai'][i], eval_data['English'][i]])
#     writer.writerow(['translate english to thai', eval_data['English'][i], eval_data['Thai'][i]])

# pd.read_csv('/content/train.csv')

In [5]:
import pandas as pd

train_data = pd.read_csv('/content/gdrive/MyDrive/train.csv')
train_data

,prefix,input_text,target_text
0,translate thai to english,การศึกษานี้ชี้ให้เห็นว่าปฏิสัมพันธ์ทางสังคมของ...,This study indicates that the social interacti...
1,translate english to thai,This study indicates that the social interacti...,การศึกษานี้ชี้ให้เห็นว่าปฏิสัมพันธ์ทางสังคมของ...
2,translate thai to english,ซึ่งประกอบด้วย 1. ผู้ใช้งานทั่วไป เช่น นักศึกษ...,"Which consists of 1. general users, such as st..."
3,translate english to thai,"Which consists of 1. general users, such as st...",ซึ่งประกอบด้วย 1. ผู้ใช้งานทั่วไป เช่น นักศึกษ...
4,translate thai to english,การทดสอบประสิทธิภาพแสดงระดับความร้อนเฉลี่ยของเ...,The performance test shows the average heat le...
...,...,...,...
8495,translate english to thai,The objective of this study is to select appro...,การศึกษาในครั้งนี้ มีวัตถุประสงค์เพื่อคัดเลือก...
8496,translate thai to english,ผลการทดลองพบว่าการผลิตไบโอดีเซลจากน้ำมันปาล์มบ...,The experimental results found that biodiesel ...
8497,translate english to thai,The experimental results found that biodiesel ...,ผลการทดลองพบว่าการผลิตไบโอดีเซลจากน้ำมันปาล์มบ...
8498,translate thai to english,ผลการวิจัยพบว่าฝาครอบเครื่องมือถูลดข้อบกพร่องเ...,The results of the research showed that the Ru...


In [6]:
eval_data = pd.read_csv('/content/gdrive/MyDrive/eval.csv')
eval_data

,prefix,input_text,target_text
0,translate thai to english,ผู้วิจัยมีวัตถุประสงค์ 3 ประการ 1. เพื่อศึกษาข...,The researcher had 3 objectives 1. to study ge...
1,translate english to thai,The researcher had 3 objectives 1. to study ge...,ผู้วิจัยมีวัตถุประสงค์ 3 ประการ 1. เพื่อศึกษาข...
2,translate thai to english,ค่าเฉลี่ยห้าวันของประสิทธิภาพสูงสุดในแต่ละวันค...,The five days average of the maximum efficienc...
3,translate english to thai,The five days average of the maximum efficienc...,ค่าเฉลี่ยห้าวันของประสิทธิภาพสูงสุดในแต่ละวันค...
4,translate thai to english,จากนั้นวิเคราะห์ผลแบบสอบถามทางด้านความพึงพอใจ ...,"Then, data are analyzed for three parts, which..."
...,...,...,...
1495,translate english to thai,"By the used of Fourier series, it is shown tha...",จากการใช้อนุกรมฟูริเยร์ แสดงให้เห็นว่าสามารถใช...
1496,translate thai to english,ปัจจัยการทดสอบ 3 ปัจจัย ได้แก่ น้ำหนักของกากกา...,Three testing factors were identified weight o...
1497,translate english to thai,Three testing factors were identified weight o...,ปัจจัยการทดสอบ 3 ปัจจัย ได้แก่ น้ำหนักของกากกา...
1498,translate thai to english,ผลการทดลองพบว่าแตงที่ปลูกในดินด่างผสมปุ๋ยหมักอ...,The results showed that the melons grown in an...


In [7]:
import logging
import torch
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_df = pd.read_csv('/content/gdrive/MyDrive/train.csv').astype(str)
eval_df = pd.read_csv('/content/gdrive/MyDrive/eval.csv').astype(str)

train_df["prefix"] = ""
eval_df["prefix"] = ""

model_args = T5Args()
model_args.max_seq_length = 128
model_args.train_batch_size = 8
model_args.eval_batch_size = 8
model_args.num_train_epochs = 9
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5 Thai-English Translation"

device = torch.cuda.is_available()
model = T5Model("mt5", "google/mt5-small", args=model_args, use_cuda=device)

In [8]:
# !wandb login --relogin

In [9]:
!rm -rf '/content/outputs'
!rm -rf '/content/runs'
!rm -rf '/content/wandb'

# Train the model
model.train_model(train_df, eval_data=eval_df)

  0%|          | 0/8500 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3707: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprec

Epoch:   0%|          | 0/9 [00:00<?, ?it/s]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: defymiygfl (sigmarai). Use `wandb login --relogin` to force relogin


Running Epoch 0 of 9:   0%|          | 0/1063 [00:00<?, ?it/s]

...

Running Epoch 7 of 9:   0%|          | 0/1063 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

Running Epoch 8 of 9:   0%|          | 0/1063 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

(9567,
 {'global_step': [1063, 2126, 3189, 4252, 5315, 6378, 7441, 8504, 9567],
  'eval_loss': [2.865188219445817,
   2.286305175182667,
   2.1057690594424594,
   1.9929383194192927,
   1.9388758825494887,
   1.8928963545155018,
   1.8977596667852807,
   1.9354597116087346,
   1.9440951727806253],
  'train_loss': [3.9341776371002197,
   3.2587904930114746,
   2.259096384048462,
   2.1589395999908447,
   2.113875389099121,
   1.2871503829956055,
   1.3796052932739258,
   0.8972505331039429,
   0.9907583594322205]})

In [10]:
# Evaluate the model
result = model.eval_model(eval_df)

  0%|          | 0/1500 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/188 [00:00<?, ?it/s]

In [11]:
import logging
import sacrebleu
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = T5Args()
model_args.max_length = 96
model_args.length_penalty = 1
model_args.num_beams = 10

model = T5Model("mt5", "/content/outputs", args=model_args)

In [12]:
valid = pd.read_csv("/content/gdrive/MyDrive/eval.csv").astype(str)

thai_truth = [valid.loc[valid["prefix"] == "translate english to thai"]["target_text"].tolist()]
to_thai = valid.loc[valid["prefix"] == "translate english to thai"]["input_text"].tolist()

english_truth = [valid.loc[valid["prefix"] == "translate thai to english"]["target_text"].tolist()]
to_english = valid.loc[valid["prefix"] == "translate thai to english"]["input_text"].tolist()

In [13]:
english_preds = model.predict(to_english)
then_bleu = sacrebleu.corpus_bleu(english_preds, english_truth)
print("Thai to English: ", then_bleu.score)

thai_preds = model.predict(to_thai)
enth_bleu = sacrebleu.corpus_bleu(thai_preds, thai_truth)
print("English to Thai: ", enth_bleu.score)

Generating outputs:   0%|          | 0/94 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/750 [00:00<?, ?it/s]

Thai to English:  28.64531564320897


Generating outputs:   0%|          | 0/94 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/750 [00:00<?, ?it/s]

English to Thai:  18.236248395500404


In [14]:
from simpletransformers.t5 import T5Model

model = T5Model("mt5", "/content/outputs")
to_predict = ['โครงงานวิศวกรรมนี้เป็นการพัฒนาระบบตรวจจับ',
              'และจําแนกโรคพืชจากภาพถ่ายใบพืชเมล่อนด้วยการเรียนรู้เชิงลึก',
              'โดยใช้เครื่อง Jetson Nano ในการประมวลผลภาพจากกล้องในโรงเรือน',
              'ซึ่งในการพัฒนาแบบจําลองการเรียนรู้เชิงลึกมีการเก็บรวบรวมภาพของใบเมล่อนจากโรงเรือน',
              'และจัดทําสัญลักษณ์ความผิดปกติของใบเมล่อน',
              'แล้วนําชุดข้อมูลที่ได้ไปใช้ในการฝึกแบบจําลองตรวจจับวัตถุในภาพ (Object Detector)',
              'ซึ่งจะสามารถตรวจพบความผิดปกติของใบเมล่อน',
              'และเมื่อระบบตรวจพบความผิดปกติจะแจ้งเตือนไปยังผู้ดูแลผ่านแอปพลิเคชันไลน์',
              'ในขั้นตอนการเลือกแบบจําลองการเรียนรู้เชิงลึกเพื่อนําไปประมวลผลบนเครื่อง Jetson Nano']
# to_predict = to_predict[0].split()
predictions = model.predict(to_predict)
for i,info in enumerate(predictions):
  print(to_predict[i] + '\n' + info + '\n')

Generating outputs:   0%|          | 0/2 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/9 [00:00<?, ?it/s]

โครงงานวิศวกรรมนี้เป็นการพัฒนาระบบตรวจจับ
This engineering project project is a development of a geographic information system.

และจําแนกโรคพืชจากภาพถ่ายใบพืชเมล่อนด้วยการเรียนรู้เชิงลึก
And classification of plant diseases from a chrysanthemum leaves with

โดยใช้เครื่อง Jetson Nano ในการประมวลผลภาพจากกล้องในโรงเรือน
The using the jetson motor machine to processing the image from the houses in the

ซึ่งในการพัฒนาแบบจําลองการเรียนรู้เชิงลึกมีการเก็บรวบรวมภาพของใบเมล่อนจากโรงเรือน
Which in the development of a graphic learning model, the images collecting of

และจัดทําสัญลักษณ์ความผิดปกติของใบเมล่อน
And prepare the images of the physical characteristics of the stalks.

แล้วนําชุดข้อมูลที่ได้ไปใช้ในการฝึกแบบจําลองตรวจจับวัตถุในภาพ (Object Detector)
Then, the dataset was used to deploy the simulation model to detect the object

ซึ่งจะสามารถตรวจพบความผิดปกติของใบเมล่อน
Which can be easily detect the dysfunction of a palm branch.

และเมื่อระบบตรวจพบความผิดปกติจะแจ้งเตือนไปยังผู้ดูแลผ่านแอปพ